<a href="https://colab.research.google.com/github/kty0307/Blog/blob/main/GPTs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. 환경설정

In [59]:
import requests
import json
import pandas as pd

In [60]:
class TLSAdapter(requests.adapters.HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("AES128-SHA256")
        kwargs["ssl_context"] = ctx
        return super(TLSAdapter, self).init_poolmanager(*args, **kwargs)

# 1. 저축은행

## 1.1. 신용점수별 금리

In [50]:
def savings_cs(year, month):
  url = "https://www.fsb.or.kr/ratloanconf_0200.jct"
  payload = {
      "_JSON_": json.dumps({
          "SORT_COLUMN": "",
          "SORT": "",
          "PRE_MONTH_MONEY": "",
          "SUBMIT_MONTH": f"{year:04d}{month:02d}"
      })
  }

  response = requests.post(url, data=payload)

  data = response.text

  json_data = json.loads(data)

  # 데이터 추출 및 DataFrame 생성
  df = pd.DataFrame(json_data['REC'])
  df = df[['BANK_NAME', 'SUBMIT_MONTH', 'A_RATE1_3', 'A_RATE1', 'A_RATE2',
            'A_RATE3', 'A_RATE_AVE']]

  return df


crawled_data = savings_cs(2024, 12)

crawled_data

,BANK_NAME,SUBMIT_MONTH,A_RATE1_3,A_RATE1,A_RATE2,A_RATE3,A_RATE_AVE
0,BNK,202412,12.97,13.7,14.83,15.13,14.31
1,DB,202412,11.55,11.71,12.1,NaN,11.91
2,IBK,202412,12.63,14.27,16.01,NaN,15.29
3,JT,202412,13.57,13.86,14.04,13.83,13.97
4,JT친애,202412,13.92,14.86,15.22,15.3,15.17
5,KB,202412,12.21,13.2,14.68,16.19,14.27
6,NH,202412,11.13,11.64,13.02,13.52,12.41
7,OK,202412,14.21,15.05,16.75,17.94,16.46
8,OSB,202412,14.73,15.42,16.11,18.01,15.93
9,SBI,202412,14.05,14.7,15.15,15.86,15.14


## 1.2. 금리대별 취급비중

In [53]:
def savings_int(year, month):
    url = "https://www.fsb.or.kr/ratloanconf_0300.jct"

    payload = {
        "SORT": "",
        "SUBMIT_MONTH": f"{year}{month:02}"
    }

    response = requests.post(url, data={"_JSON_": json.dumps(payload)})
    response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    df = pd.DataFrame(data["REC"])  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    desired_columns = ["BANK_NAME", "SUBMIT_MONTH", "HANDING_WEIGHT_10", "HANDING_WEIGHT_12", "HANDING_WEIGHT_14"
                        ,"HANDING_WEIGHT_16", "HANDING_WEIGHT_18", "HANDING_WEIGHT_20"]
    df = df[desired_columns] #원하는 열만 남기기

    return df

# 예시 사용

data = savings_int(2024, 12)

data

,BANK_NAME,SUBMIT_MONTH,HANDING_WEIGHT_10,HANDING_WEIGHT_12,HANDING_WEIGHT_14,HANDING_WEIGHT_16,HANDING_WEIGHT_18,HANDING_WEIGHT_20
0,BNK,202412,0.44,16.82,25.54,46.61,10.59,0
1,DB,202412,NaN,66.42,28.2,3.91,1.47,NaN
2,IBK,202412,NaN,9.23,36.05,14.95,12.6,27.17
3,JT,202412,35.07,17,36.23,6.88,4.82,NaN
4,JT친애,202412,NaN,NaN,33.06,42.01,16.68,8.25
5,KB,202412,5.18,24.26,24.11,16.61,12.85,16.99
6,NH,202412,0.09,44.58,32.35,18.56,3.38,1.04
7,OK,202412,NaN,5.68,9.36,27.59,28.89,28.48
8,OSB,202412,NaN,NaN,23.81,33.23,11.72,31.24
9,SBI,202412,0.54,8.64,24.48,33.08,19.18,14.08


# 2. 신용카드

## 2.1. 카드론_신용점수별 금리

In [62]:
def card_loan_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditcard/creditcardDisclosureDetail25Ajax"

    # cgc_seq(월)을 찾기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": 1458,
        "cgcMode": 25,
        "cgcYyyy": year,
        "mcSeq": []
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['cgcquarter'] == month:
            cgc_seq = item['cgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": cgc_seq,
        "cgcMode": 25,
        "cgcYyyy": year,
        "mcSeq": [31, 96, 1, 106, 14, 13, 12, 98, 502, 108, 619, 11, 97, 105, 103, 22]
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "cgcSeq", "cgCardPoint1", "cgCardPoint2", "cgCardPoint3"
                ,"cgCardPoint4", "cgCardPointAvg"]] #원하는 열만 남기기

    df["cgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    return df

# 예시 사용

data = card_cs(2024, 11)

data

,mcCompany,cgcSeq,cgCardPoint1,cgCardPoint2,cgCardPoint3,cgCardPoint4,cgCardPointAvg
0,경남은행,202411,8.08,12.20,15.20,17.46,12.20
1,광주은행,202411,12.59,14.04,15.60,16.92,14.94
2,롯데카드,202411,11.10,12.86,15.96,18.02,14.93
3,부산은행,202411,11.04,12.11,14.01,15.59,12.33
4,비씨카드,202411,11.79,13.19,14.78,14.96,13.30
5,삼성카드,202411,12.25,13.04,15.51,18.00,14.79
6,신한카드,202411,11.09,12.78,15.00,17.19,14.46
7,씨티은행,202411,8.89,11.48,12.45,14.34,12.09
8,우리카드,202411,10.07,13.72,16.51,18.83,15.39
9,전북은행,202411,0.00,0.00,14.54,17.77,14.64


## 2.2. 현금서비스_신용점수별 금리

In [70]:
def card_cash_cs(year, month):
    url = "https://gongsi.crefia.or.kr/portal/creditcard/creditcardDisclosureDetail20Ajax"

    # cgc_seq(월)을 찾기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": 1460,
        "cgcMode": 20,
        "cgcYyyy": year,
        "mcSeq": []
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    pick_month = json.loads(response.text)  # 응답 문자열 반환

    for item in pick_month['configListMm']:
        if item['cgcquarter'] == month:
            cgc_seq = item['cgcSeq']
            break  # 첫 번째로 찾은 값만 출력하고 반복문 종료

    # 실제 데이터를 추출하기 위한 크롤링
    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
        "cgcSeq": cgc_seq,
        "cgcMode": 20,
        "cgcYyyy": year,
        "mcSeq": [31, 96, 1, 106, 14, 13, 12, 98, 502, 108, 619, 11, 97, 105, 103, 22]
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    data = json.loads(response.text)

    pick_data = data["resultList"]
    df = pd.DataFrame(pick_data)  # 모든 열을 포함하는 DataFrame 생성

    # 원하는 열만 선택
    df = df[["mcCompany", "cgcSeq", "cgMoneyPoint1", "cgMoneyPoint2", "cgMoneyPoint3"
                ,"cgMoneyPoint4", "cgMoneyPointAvg"]] #원하는 열만 남기기

    df["cgcSeq"] = f"{year}{month:02}" if month < 10 else f"{year}{month}"

    return df

# 예시 사용

data = card_cash_cs(2024, 11)

data

,mcCompany,cgcSeq,cgMoneyPoint1,cgMoneyPoint2,cgMoneyPoint3,cgMoneyPoint4,cgMoneyPointAvg
0,경남은행,202411,14.23,16.88,17.72,18.37,17.81
1,광주은행,202411,16.27,17.10,17.78,18.72,18.33
2,롯데카드,202411,15.30,16.50,17.96,19.16,18.50
3,부산은행,202411,16.34,17.48,18.26,18.37,17.94
4,비씨카드,202411,14.76,16.55,18.23,19.33,18.44
5,삼성카드,202411,14.33,15.25,17.37,19.01,17.81
6,신한카드,202411,13.75,15.08,16.95,18.67,17.66
7,씨티은행,202411,13.02,17.66,19.58,19.85,19.51
8,우리카드,202411,16.09,17.25,18.81,19.45,18.92
9,전북은행,202411,14.29,17.04,18.71,19.31,18.82


# 3. 캐피탈

## 3.1. 신용점수별 금리

In [72]:
def capital_cs(clgc_mode, card_item, clgc_seq, clgc_yyyy):
    url = "https://gongsi.crefia.or.kr/portal/creditloan/creditloanDisclosureDetail11/ajax"

    with requests.session() as s:
      s.mount("https://", TLSAdapter())

      params = {
          "clgcMode": clgc_mode,
          "cardItem": card_item,
          "clgcSeq": clgc_seq,
          "clgcYyyy": clgc_yyyy
      }

      response = s.get(url, params=params)
      response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

    # pick_month = json.loads(response.text)

    return response.text




# # 함수 사용 예시
clgc_mode = 11
card_item = "1,14,13,12,502,619,103,134,39,40,623,130,41,25,156,6,55,32,58,52,61,57,64"
clgc_seq = 521
clgc_yyyy = 2024

# data = get_creditloan_disclosure(clgc_mode, card_item, clgc_seq, clgc_yyyy)

# if data:
#     print(data)
#     # 데이터 처리
# else:
#     print("데이터를 가져오는 데 실패했습니다.")

capital_cs(clgc_mode, card_item, clgc_seq, clgc_yyyy)


'{"configList":[{"clgcPubDate":"2025.01.20","clgcYear":"2025","clgcYearQuarter":"2025년 1분기","clgcSeq":524,"clgcquarter":1,"clgcYearMonth":"2025년 1월"},{"clgcPubDate":"2024.12.20","clgcYear":"2024","clgcYearQuarter":"2024년 12분기","clgcSeq":521,"clgcquarter":12,"clgcYearMonth":"2024년 12월"},{"clgcPubDate":"2024.11.20","clgcYear":"2024","clgcYearQuarter":"2024년 11분기","clgcSeq":518,"clgcquarter":11,"clgcYearMonth":"2024년 11월"},{"clgcPubDate":"2024.10.18","clgcYear":"2024","clgcYearQuarter":"2024년 10분기","clgcSeq":514,"clgcquarter":10,"clgcYearMonth":"2024년 10월"},{"clgcPubDate":"2024.09.20","clgcYear":"2024","clgcYearQuarter":"2024년 9분기","clgcSeq":511,"clgcquarter":9,"clgcYearMonth":"2024년 9월"},{"clgcPubDate":"2024.08.20","clgcYear":"2024","clgcYearQuarter":"2024년 8분기","clgcSeq":508,"clgcquarter":8,"clgcYearMonth":"2024년 8월"},{"clgcPubDate":"2024.07.19","clgcYear":"2024","clgcYearQuarter":"2024년 7분기","clgcSeq":504,"clgcquarter":7,"clgcYearMonth":"2024년 7월"},{"clgcPubDate":"2024.06.20","clgcYear

# 4. 은행

## 4.1.

SSL 오류: [SSL: WRONG_SIGNATURE_TYPE] wrong signature type (_ssl.c:1006)
